In [ ]:
from os import listdir
from os.path import isfile, join, isdir
import csv
import pandas as pd
import plotly.express as px
import datetime
path = f"/nfs/nsg/group/jacobr/SWITCH_data/router-interfaces/"

In [ ]:
directories = [dir for dir in listdir(path)]
traffic_volume_in = {}
traffic_volume_out = {}
traffic_packets_in = {}
traffic_packets_out = {}

for directory in directories:
    files = [filename for filename in listdir(join(path, directory)) if isfile(join(path, directory, filename))]
    for file in files:
        #print(file)
        if "bundle" in file or "port-channel" in file or "vlan" in file or "bidi" in file or "loopback" in file or "bvi2" in file:
            print(file)
            continue
        if len(file.split(".")) > 2:
            print(file)
            continue
        print(f"{directory, file}")
        with open(file=join(path, directory, file), mode='r') as f:
            contents = list(csv.reader(f, delimiter=","))
            for element in contents:
                if element[0] == "timestamp":
                    continue
                if element[1] == "" or element[2] == "":
                    continue
                traffic_volume_in[element[0]] = traffic_volume_in.get(element[0],0) + float(element[1])
                traffic_volume_out[element[0]] = traffic_volume_out.get(element[0],0) + float(element[2])

                if len(element) < 7:
                    continue
                if element[5] == "" or element[6] == "":
                    continue
                
                traffic_packets_in[element[0]] = traffic_packets_in.get(element[0],0) + float(element[5])
                traffic_packets_out[element[0]] = traffic_packets_out.get(element[0],0) + float(element[6])


In [ ]:
power_path = f"/nfs/nsg/group/jacobr/router-power-dataset/cisco/"
total_power = {}

power_directories = [dir for dir in listdir(power_path)]
print(power_directories)
for directory in power_directories:
    locations = [filename for filename in listdir(join(power_path, directory))]
    for location in locations:
        if "metadata" in location:
            continue
        with open(file=join(power_path, directory, location,"PSU/total.csv"), mode='r') as f:
            contents = list(csv.reader(f, delimiter=","))
            for element in contents:
                if element[0] == "timestamp":
                    continue
                total_power[element[0]] = total_power.get(element[0],0) + float(element[3])

In [ ]:
with open("./traffic_volume_power.csv", mode='w') as f:
    f.write("timestamp,traffic volume in [Bytes],traffic volume out [Bytes],traffic volume [Bytes],packets in,packets out,packets,power [W]\n")
    for key in sorted(traffic_volume_in.keys()):
        if key not in traffic_volume_out:
            print(key)
        if key not in total_power:
            print(key)
            continue
        f.write(f"{datetime.datetime.fromtimestamp(int(key))},{traffic_volume_in[key]},{traffic_volume_out[key]},{traffic_volume_in[key]+traffic_volume_out[key]}," +
                f"{traffic_packets_in[key]},{traffic_packets_out[key]},{traffic_packets_in[key]+traffic_packets_out[key]},{total_power[key]}\n")

In [ ]:
traffic = pd.read_csv("./traffic_volume_power.csv")
traffic["traffic volume [100s of Gbit]"] = traffic["traffic volume [Bytes]"] * 8 / 10**11
traffic["power [kW]"] = traffic["power [W]"] / 10**3
px.line(traffic, x="timestamp", y=["traffic volume [100s of Gbit]", "power [kW]", "packets"], title="Traffic Volume and Power Consumption").show()